In [1]:
# Se importan las librerías a utilizar
from pymoo.optimize import minimize
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.core.problem import Problem
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.repair.rounding import RoundingRepair
from pymoo.operators.sampling.rnd import IntegerRandomSampling

In [2]:
# Se crea una clase llamada "MyProblem" que será la que contiene los parámetros evolutivos del algoritmo 
class MyProblem(Problem):

    def __init__(self):
        # Se define el número de variables de entrada, cantidad de objetivos a evaluar y valores máximos y mínimos que pueden tener los individuos 
        super().__init__(n_var=3, n_obj=2, n_ieq_constr=0, xl=np.array([0, 0, 0]), xu=np.array([50, 50, 50]), vtype=int)

    def _evaluate(self, x, out, *args, **kwargs):
        # Se crea la población inicial
        poblacion = np.round(x).astype(int)
        To, Fr, M = poblacion[:, 0], poblacion[:, 1], poblacion[:, 2]
        
        # Se define un castigo para los individuos que utilicen más materiales de los que se tienen en bodega según las instrucciones de la tarea
        castigo=np.zeros(len(Fr))
        for i in range (len(Fr)):
            c=2*To[i]+Fr[i]+5*M[i]
            m=6*To[i]+3*Fr[i]+12*M[i]
            ca=2*To[i]+8*Fr[i]+4*M[i]
            ac=2*To[i]+6*Fr[i]+6*M[i]
            if c>280:
                castigo[i]=20000
            elif m>120:
                castigo[i]=20000
            elif ca>670:
                castigo[i]=20000
            elif ac>250:
                castigo[i]=20000
            
        # Codificación de las funciones de calida, la ganancia (monetaria) es una función que se busca maximizar, la producción se busca minimizar,
        # ya que esta función evalúa si se manufactura una cantidad similar de máquinas, de forma que si esta función da cero, estamos frente 
        # a un caso donde se tiene la misma cantidad de cada una de las tres máquinas
        ganancia = -(4*To+8*Fr+12*M)+ castigo
        produccion =(abs(To-Fr) + abs(To-M) + abs(Fr-M))+castigo
        out["F"] = np.column_stack([produccion, ganancia])
#     
problem = MyProblem()

In [3]:
# Se definen los parámetros del algoritmo genético (número de individuos por generación, probabilidad de mutación, cantidad de generaciones qeu se
# evalúan y tipo de cruzamiento
results = minimize(
            problem = problem,
            algorithm=NSGA2(pop_size=40,
                            sampling=IntegerRandomSampling(),
                            crossover=SBX(prob=1, vtype=float, repair=RoundingRepair()),
                            mutation=PM(prob=1.5, vtype=float, repair=RoundingRepair()),
                            eliminate_duplicates=True), 
            termination=('n_gen', 600),
            seed=1,
            save_history=True
        )

In [5]:
# Impresión de la curva de Pareto y de los mejores individuos para su selección
resultados = results.F.T
fig = go.Figure(data=go.Scatter(x=resultados[0], y=resultados[1], mode='markers'))
fig.show()
print(results.X)
print(results.F)

[[ 4 15  4]
 [ 2 16  5]
 [ 5 14  4]
 [ 4 16  4]
 [ 2 20  4]
 [ 5  9  5]
 [ 5  5  5]
 [ 3 10  6]
 [ 5 10  5]
 [ 3 21  3]
 [ 4 20  3]
 [ 3 22  3]
 [ 4 11  5]
 [ 1 26  3]
 [ 6  8  5]
 [ 3 26  2]
 [ 3 14  5]
 [ 4 12  5]
 [ 2 23  3]
 [ 2 27  2]
 [ 1 29  2]
 [ 3 18  4]
 [ 5  6  6]
 [ 1 34  1]
 [ 3 17  4]
 [ 1 22  4]
 [ 0 35  1]
 [ 1 33  1]
 [ 4  8  6]
 [ 1 30  2]
 [ 0 36  1]
 [ 1 38  0]
 [ 0 39  0]
 [ 0 40  0]
 [ 2 32  1]
 [ 0 32  2]
 [ 2 28  2]
 [ 0 28  3]
 [ 2 24  3]]
[[  22. -184.]
 [  28. -196.]
 [  20. -180.]
 [  24. -192.]
 [  36. -216.]
 [   8. -152.]
 [   0. -120.]
 [  14. -164.]
 [  10. -160.]
 [  36. -216.]
 [  34. -212.]
 [  38. -224.]
 [  14. -164.]
 [  50. -248.]
 [   6. -148.]
 [  48. -244.]
 [  22. -184.]
 [  16. -172.]
 [  42. -228.]
 [  50. -248.]
 [  56. -260.]
 [  30. -204.]
 [   2. -140.]
 [  66. -288.]
 [  28. -196.]
 [  42. -228.]
 [  70. -292.]
 [  64. -280.]
 [   8. -152.]
 [  58. -268.]
 [  72. -300.]
 [  76. -308.]
 [  78. -312.]
 [  80. -320.]
 [  62. -276.]
 [  64